In [0]:
'''
  let's try and train a GRU here...

  resources: https://blog.floydhub.com/gru-with-pytorch/ 

  TODO NEXT: modularize this code?
'''


In [61]:
#first, fetch the dataset (headlines + stock movements)
import pickle as pkl

fpath = './drive/My Drive/Colab Notebooks/all_news_trdata/allstocks_trdata_8k.pkl'
with open(fpath, 'rb') as f:
  trdata = pkl.load(f)

print(trdata.shape)
print(type(trdata))
print(trdata)

(7489, 2)
<class 'numpy.ndarray'>
[['Benghazi Committee Releases Final Report, Slams Clinton. \n In American Markets, Panic Begins to Subside - The New York Times. \n Barack Obama: ‘Mr. Trump Embodies Global Elites’ Not Working Class - Breitbart. \n Reactions to the Supreme Court Ruling on Texas’ Abortion Law - The New York Times. \n Stacey Dash Talks ’Dash America’ Movement: Conservatives Can ’Take Back Influence and Power’ in Hollywood - Breitbart'
  '1']
 ['‘Brexit’ Opens Uncertain Chapter in Britain’s Storied History - The New York Times. \n Benghazi Committee Releases Final Report, Slams Clinton. \n The 15 Questions About Benghazi Barack Obama Does Not Want To Answer - Breitbart. \n Jenner on ’SI’ 40 Years Later: My ‘Macho Male’ Olympics Body Disguised ‘The Woman Living Inside Me’ - Breitbart. \n Sharyl Attkisson: Obama and Clinton Lied to the Public, ‘Impeded the Investigation’ - Breitbart'
  '1']
 ['Hamas Ends Week-Old Deal Importing Israeli Watermelons Into Gaza. \n Demand for 

In [0]:
import os
import sys

#run file with all dependencies.
%run './drive/My Drive/Colab Notebooks/news_fns.py'

titles = trdata[:,0]
labels = trdata[:,1]
train_loader, val_loader, titles_tok_padded, title_lens, hash_size = create_dataset_dataloader(titles, labels)

#need hash_size when initializing gru...

In [0]:
import torch
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence

'''
[this is all working fine]
source: https://blog.floydhub.com/gru-with-pytorch/
pytorch documentation (not very good): https://pytorch.org/docs/master/nn.html#gru
'''


def create_emb_layer(num_embeddings, embedding_dim, emb_wts = None, non_trainable=False):
    if emb_wts is None:
        embedding = nn.Embedding(num_embeddings, embedding_dim)
    else:
        num_embeddings, embedding_dim = emb_wts.shape
        embedding = nn.Embedding(num_embeddings, embedding_dim)
        embedding.load_state_dict(emb_wts)
        if non_trainable:
            embedding.weight.requires_grad = False
    return embedding

#create a GRU class
class GRUNet(nn.Module):
    
    '''
        input parameters:
            input_dim (gru): size of the input embeddings (~50?)
            hidden_dim (gru/affine): size of the hidden parameter (50) = output of the gru = input to the affine layer.
            output_dim (affine): size of the output (2) = final num_classes
            n_layers_gru: number of stacked layers of the GRU (1 or 2) ~ how to make bidirectional?
            
            embedding_dim (embed): should be the same as input_dim (size of the)
            emb_wts (embed): matrix of pretrained embeddings (GloVe)
            num_embeddings (embed): total vocab size...
    '''
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers_gru, drop_prob=0.2,
                num_embeddings = 0, embedding_dim = 0, emb_wts = None, non_trainable=False):
        super(GRUNet, self).__init__() #initialize the super-class?
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers_gru
        
        #we need an embedding layer, a gru layer, and maybe a FC layer
        #create an embedding layer here.
        self.embedding = create_emb_layer(num_embeddings, embedding_dim, emb_wts, non_trainable)             
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers_gru, batch_first=True, dropout=drop_prob) #needs batchfirst??
        self.relu = nn.ReLU()
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    #todo: rewrite this to remove padding
    def forward(self, x, lens, h):
        out = self.embedding(x)
        #remove padding
        x_packed = pack_padded_sequence(out, lens, batch_first=True, enforce_sorted=False)
        out, h = self.gru(x_packed, h)
        #VERY IMPORTANT: hidden dimension (h) is the final output of every sequence!
        out = self.relu(h)
        out = self.fc(out)
        return out
    
    def init_hidden(self, batch_size, device='cpu'):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        return hidden

    

In [100]:
# Create a training & validation procedure:

'''
  training:
    - load a batch
    - do a forward pass
    - do a backward pass with the given "criterion" loss
      ~ remember you have to detach gradients ***
      ~ remember you have to zero_gradient
    
  validation:
    - load & run all batches
    - calculate the validation accuracy after every epoch.
'''

import time

# ========================================
#               Validation
# ========================================

#given logits and labels, return an accuracy
def flat_accuracy(logits, labels):
  labels = labels.numpy()
  preds = logits.detach().numpy()
  pred_flat = np.argmax(preds, axis=1).flatten()
  #print("\npreds:")
  #print(pred_flat)
  #print(labels)
  #print(np.sum(pred_flat == labels))
  return 1.0 * np.sum(pred_flat == labels) / len(labels)

def validate(model, data_loader):
  criterion = nn.CrossEntropyLoss()
  model.eval() #put model in evaluation mode...
  eval_loss, eval_acc, ctr = 0,0,0
  for batch in data_loader:
    titles, lengths, labels = batch['title'], batch['length'], batch['label']
    bsize = titles.shape[0]
    h0 = model.init_hidden(bsize)
    logits = model(titles, lengths, h0) #also add lengths to do pack_padded_seq

    tl = torch.tensor([0 if l=='-1' else 1 for l in labels]) ##TODO: FIX IN PREPROCESSING. Convert to integers, either 0 or 1
    loss = criterion(logits.view(bsize, 2), tl)
    acc = flat_accuracy(logits.view(bsize, 2), tl)
    eval_loss += loss
    eval_acc += acc
  #return final accuracy
  return eval_loss/len(data_loader), eval_acc/len(data_loader)
  



# ========================================
#               Training
# ========================================

#earlier variables needed: hash_size, learn_rate
#utilize the adam optimizer...

#initialize model
input_dim, hidden_dim, num_classes = 50, 50, 2
n_layers_gru = 1

learn_rate = 0.001
num_epochs = 2  #let's say between 2 & 4.

gru = GRUNet(input_dim, hidden_dim, num_classes, n_layers_gru, 0.2, hash_size, 50)

#loss (https://pytorch.org/docs/stable/nn.html#crossentropyloss)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gru.parameters(), lr = learn_rate)


gru.train()
start_time = time.clock()  
for epoch in range(num_epochs):
  cum_loss = 0.0
  counter = 0

  for batch in train_loader:
    titles, lengths, labels = batch['title'], batch['length'], batch['label']
    bsize = titles.shape[0]

    #zero grad (??)
    gru.zero_grad()

    #run a forward pass
    h0 = gru.init_hidden(bsize)
    logits = gru(titles, lengths, h0) #also add lengths to do pack_padded_seq

    tl = torch.tensor([0 if l=='-1' else 1 for l in labels]) ##TODO: FIX IN PREPROCESSING. Convert to integers, either 0 or 1
    loss = criterion(logits.view(bsize, 2), tl)

    #back pass
    loss.backward()

    cum_loss += loss.item()
    counter += 1
    if (counter % 50) == 0:
      print("Epoch {}........ Step {}/{}............ Avg Loss: {}".format(epoch, counter, len(train_loader), cum_loss/counter))
  
  ### at the end of the epoch, compute validation accuracy
  val_loss, val_acc = validate(gru, val_loader)
  el_time = time.clock() - start_time
  print("Epoch {}/{} done, validation accuracy: {}, elapsed time: {}".format(epoch+1, num_epochs, val_acc, el_time))

    




/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch 0........ Step 50/211............ Avg Loss: 0.698989679813385
Epoch 0........ Step 100/211............ Avg Loss: 0.6994273614883423
Epoch 0........ Step 150/211............ Avg Loss: 0.7005022303263346
Epoch 0........ Step 200/211............ Avg Loss: 0.7007146316766739
Epoch 1/2 done, validation accuracy: 0.4282852564102564, elapsed time: 16.717280999999986
Epoch 1........ Step 50/211............ Avg Loss: 0.698989679813385
Epoch 1........ Step 100/211............ Avg Loss: 0.6994273614883423
Epoch 1........ Step 150/211............ Avg Loss: 0.7005022303263346
Epoch 1........ Step 200/211............ Avg Loss: 0.7007146316766739
Epoch 2/2 done, validation accuracy: 0.4282852564102564, elapsed time: 33.413320999999996


In [99]:
'''
val_loss, val_acc = validate(gru, val_loader)
el_time = time.clock() - start_time
print("Epoch {}/{} done, validation accuracy: {}, elapsed time: {}".format(epoch+1, num_epochs, val_acc, el_time))
'''

#given logits and labels, return an accuracy
def flat_accuracy(logits, labels):
  labels = labels.numpy()
  preds = logits.detach().numpy()
  pred_flat = np.argmax(preds, axis=1).flatten()
  #print("\npreds:")
  #print(pred_flat)
  #print(labels)
  #print(np.sum(pred_flat == labels))
  return 1.0 * np.sum(pred_flat == labels) / len(labels)

def validate(model, data_loader):
  criterion = nn.CrossEntropyLoss()
  model.eval() #put model in evaluation mode...
  eval_loss, eval_acc, ctr = 0,0,0
  for batch in data_loader:
    titles, lengths, labels = batch['title'], batch['length'], batch['label']
    bsize = titles.shape[0]
    h0 = model.init_hidden(bsize)
    logits = model(titles, lengths, h0) #also add lengths to do pack_padded_seq

    tl = torch.tensor([0 if l=='-1' else 1 for l in labels]) ##TODO: FIX IN PREPROCESSING. Convert to integers, either 0 or 1
    loss = criterion(logits.view(bsize, 2), tl)
    acc = flat_accuracy(logits.view(bsize, 2), tl)
    eval_loss += loss
    eval_acc += acc
  #return final accuracy
  return eval_loss/len(data_loader), eval_acc/len(data_loader)

print(type(gru))
print(type(val_loader))
loss, acc = validate(gru, val_loader)
print(loss)
print(acc)

<class '__main__.GRUNet'>
<class 'torch.utils.data.dataloader.DataLoader'>
tensor(0.7323, grad_fn=<DivBackward0>)
0.4476161858974359


In [94]:
pred_flat

NameError: ignored

In [74]:
print(type(labels))
print(type(labels[0]))
print(logits.size())

l0 = [int(l) for l in labels]
print(torch.tensor(l0))

<class 'list'>
<class 'numpy.str_'>
torch.Size([1, 32, 2])
tensor([-1,  1,  1,  1,  1, -1,  1, -1, -1, -1, -1,  1, -1, -1,  1,  1, -1, -1,
        -1,  1,  1,  1,  1,  1, -1,  1, -1, -1,  1, -1,  1, -1])


In [50]:
# step by step, "informal" journey through the neural net.
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import one_hot
import numpy as np

from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pack_padded_sequence


### first, start by preprocessing & creating input
y = [0, 1]
x_sentlist = ["a quick brown fox jumps", "a lazy brown dog barks loudly"]
allwds = '. '.join(x_sentlist)
vocab_size = len(set( text_to_word_sequence(allwds) ))
hash_size = vocab_size * 10

#first, tokenize x (start w/ batchsize 2)
batch_size=2
input_sents = []
for sent in x_sentlist:
  input_sents.append(torch.tensor(one_hot(sent, hash_size)))
print(input_sents)

# add padding & convert to torch tensor
lens = [len(s) for s in input_sents]
sents_tok_padded = pad_sequence(input_sents, batch_first=True, padding_value=0)
print("with padding included...")
print(sents_tok_padded)

x = torch.tensor(sents_tok_padded)


#---------------------------------------------#
### next, put the input x into the GRU net.

print("inputs...")
print(x)
print(lens)
print(hash_size)

#initialize "neural net" layer
emb_size, hidden_dim, drop_prob = 50, 50, 0.2
embedding = nn.Embedding(hash_size, emb_size)
gru_lyr = nn.GRU(emb_size, hidden_dim, 1, batch_first=True, dropout=drop_prob) #needs batchfirst??
fc_lyr = nn.Linear(hidden_dim, 2)


### let's start the forward prop...

x_emb = embedding(x)
#print(x_emb)
print(x_emb.shape) #2 x 6 x 50

#run this through a pack_padded_seq operator (this output is correct, it just doesn't look correct)
p0 = pack_padded_sequence(x_emb, lens, batch_first=True, enforce_sorted=False)
x_packed, b, _, _ = p0
print(x_packed)
#print(x_packed[0].shape)

### very important note: hf = the final output for each sequence!!! (we need to pass this on!)
gru_op, hf = gru_lyr(p0)

#pass the final output through an affine layer
output = fc_lyr(hf)

print(output)


inputs...
tensor([[78, 13, 52, 40, 19,  0],
        [78, 49, 52,  5, 81, 61]])
[5, 6]
35105
torch.Size([2, 6, 50])
tensor([[ 1.0150e+00, -1.1883e+00,  4.2255e-01,  8.1253e-01,  1.2233e-01,
         -3.1855e-01, -8.8173e-01, -4.9370e-01,  1.4209e+00, -2.5932e-01,
          1.4959e+00, -9.0945e-01, -1.8885e-01, -2.7176e+00, -9.1975e-01,
          1.1334e+00, -3.9781e-01,  2.9156e-01, -1.0414e+00,  8.4001e-01,
          7.5788e-02,  1.2013e+00,  4.7893e-01, -5.8157e-01,  7.9253e-01,
          1.4535e+00, -3.4666e-01,  7.5556e-01,  6.6904e-02, -8.8769e-01,
          8.2614e-01, -2.0586e+00,  8.0433e-01,  6.8744e-01, -1.0236e-01,
          1.3137e+00,  3.2567e-01, -1.1686e+00,  2.5810e-01,  1.8774e-01,
          1.2054e+00, -9.0501e-01, -1.4245e+00, -1.1421e+00, -1.1427e+00,
          4.4023e-01, -9.0785e-01, -7.0045e-01, -1.0885e+00,  9.2201e-02],
        [ 1.0150e+00, -1.1883e+00,  4.2255e-01,  8.1253e-01,  1.2233e-01,
         -3.1855e-01, -8.8173e-01, -4.9370e-01,  1.4209e+00, -2.5932e-

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [54]:
#next, pass the output into a fc layer
output = fc_lyr(hf)
print(output)

# eureka: hf is the final output (which is what we needed)
'''
print(hf.shape)
print(hf)
'''
#unpack the packed sequence
'''from torch.nn.utils.rnn import pad_packed_sequence
x_padded, sent_lens = pad_packed_sequence(x_packed, batch_first=True, padding_value=0.0)
print(x_padded.shape)
print(sent_lens)
print(x_padded[0][0])
'''

tensor([[[ 0.1547,  0.1246],
         [-0.0638,  0.1120]]], grad_fn=<AddBackward0>)


'from torch.nn.utils.rnn import pad_packed_sequence\nx_padded, sent_lens = pad_packed_sequence(x_packed, batch_first=True, padding_value=0.0)\nprint(x_padded.shape)\nprint(sent_lens)\nprint(x_padded[0][0])\n'